# (experimental) Dynamic Quantization on BERT

**Author**: [Jianyu Huang](https://github.com/jianyuh)

**Reviewed by**: [Raghuraman Krishnamoorthi](https://github.com/raghuramank100)

**Edited by**: [Jessica Lin](https://)

# Introduction


In this tutorial, we will apply the dynamic quantization on a BERT
model, closely following the BERT model from [the HuggingFace
Transformers examples](https://github.com/huggingface/transformers).
With this step-by-step journey, we would like to demonstrate how to
convert a well-known state-of-the-art model like BERT into dynamic
quantized model.

-  BERT, or Bidirectional Embedding Representations from Transformers,
   is a new method of pre-training language representations which
   achieves the state-of-the-art accuracy results on many popular
   Natural Language Processing (NLP) tasks, such as question answering,
   text classification, and others. The original paper can be found
   [here](https://arxiv.org/pdf/1810.04805.pdf).

-  Dynamic quantization support in PyTorch converts a float model to a
   quantized model with static int8 or float16 data types for the
   weights and dynamic quantization for the activations. The activations
   are quantized dynamically (per batch) to int8 when the weights are
   quantized to int8. In PyTorch, we have [torch.quantization.quantize_dynamic API](https://pytorch.org/docs/stable/quantization.html#torch.quantization.quantize_dynamic),
   which replaces specified modules with dynamic weight-only quantized
   versions and output the quantized model.

-  We demonstrate the accuracy and inference performance results on the
   [Microsoft Research Paraphrase Corpus (MRPC) task](https://www.microsoft.com/en-us/download/details.aspx?id=52398)
   in the General Language Understanding Evaluation benchmark [(GLUE)](https://gluebenchmark.com/). The MRPC (Dolan and Brockett, 2005) is
   a corpus of sentence pairs automatically extracted from online news
   sources, with human annotations of whether the sentences in the pair
   are semantically equivalent. As the classes are imbalanced (68%
   positive, 32% negative), we follow the common practice and report [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html).
   MRPC is a common NLP task for language pair classification, as shown
   below.

<!-- ![BERT for setence pair classification](https://drive.google.com/file/d/1m_VcRJNuMBwnrx3f0OShX6ffLyoEOJPC/view?usp=sharing). -->

![BERT for setence pair classification](https://gluon-nlp.mxnet.io/_images/bert-sentence-pair.png)


<!-- ![alt text](https://drive.google.com/file/d/1NJIWxtY39pBl0KUCOCMF5vpfuWLlSKf8/view?usp=sharing) -->








# 1. Setup
## 1.1 Install PyTorch and HuggingFace Transformers


To start this tutorial, let’s first follow the installation instructions in PyTorch [here](https://github.com/pytorch/pytorch/#installation) and HuggingFace Github Repo [here](https://github.com/huggingface/transformers#installation). In addition, we also install [scikit-learn](https://github.com/scikit-learn/scikit-learn) package, as we will reuse its built-in F1 score calculation helper function.


In [1]:
%pip install sklearn
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Because we will be using the experimental parts of the PyTorch, it is recommended to install the latest version of torch and torchvision. You can find the most recent instructions on local installation [here](https://pytorch.org/get-started/locally/). For example, to install on Mac:

## 1.2 Import the necessary modules

In this step we import the necessary Python modules for the tutorial.

In [2]:
from __future__ import absolute_import, division, print_function

import logging
import numpy as np
import os
import random
import sys
import time
import torch

from argparse import Namespace
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from tqdm import tqdm
from transformers import (BertConfig, BertForSequenceClassification, BertTokenizer,)
from transformers import glue_compute_metrics as compute_metrics
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors
from transformers import glue_convert_examples_to_features as convert_examples_to_features

# Setup logging
logger = logging.getLogger(__name__)
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.WARN)

logging.getLogger("transformers.modeling_utils").setLevel(
   logging.WARN)  # Reduce logging

print(torch.__version__)


/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.10.1+cu111


We set the number of threads to compare the single thread performance between FP32 and INT8 performance. In the end of the tutorial, the user can set other number of threads by building PyTorch with right parallel backend.

In [4]:
torch.set_num_threads(4)
print(torch.__config__.parallel_info())

ATen/Parallel:
	at::get_num_threads() : 4
	at::get_num_interop_threads() : 20
OpenMP 201511 (a.k.a. OpenMP 4.5)
	omp_get_max_threads() : 4
Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
	mkl_get_max_threads() : 4
Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
std::thread::hardware_concurrency() : 40
Environment variables:
	OMP_NUM_THREADS : [not set]
	MKL_NUM_THREADS : [not set]
ATen parallel backend: OpenMP



## 1.3 Learn about helper functions

The helper functions are built-in in transformers library. We mainly use
the following helper functions: one for converting the text examples
into the feature vectors; The other one for measuring the F1 score of
the predicted result.

The [glue_convert_examples_to_features](https://github.com/huggingface/transformers/blob/master/transformers/data/processors/glue.py) function converts the texts into input features:

-  Tokenize the input sequences;
-  Insert [CLS] in the beginning;
-  Insert [SEP] between the first sentence and the second sentence, and
   in the end;
-  Generate token type ids to indicate whether a token belongs to the
   first sequence or the second sequence.

The [glue_compute_metrics function](https://github.com/huggingface/transformers/blob/master/transformers/data/processors/glue.py) has the compute metrics with the [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html), which 
can be interpreted as a weighted average of the precision and recall,
where an F1 score reaches its best value at 1 and worst score at 0. The
relative contribution of precision and recall to the F1 score are equal.

-  The equation for the F1 score is:

F1 = 2 * (precision * recall) / (precision + recall)





## 1.4 Download the dataset

Before running MRPC tasks we download the [GLUE data](https://gluebenchmark.com/tasks) by running [this script](https://gist.github.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e) and unpack it to some directory `glue_data/MRPC`.


In [3]:
## !python download_glue_data.py --data_dir='glue_data' --tasks='MRPC' --test_labels=True
!pwd
!ls
# !python download_glue_data.py --data_dir='glue_data' --tasks='MRPC'
!ls glue_data/MRPC

/home/ty6232/DynamicQuantization_Bert
 bert_quant.pt					      glue_data
 bert_quant_state.pt				      MRPC
 download_glue_data.py				      read_model.ipynb
'(experimental)_Dynamic_Quantization_on_BERT.ipynb'
cached_dev_bert-base-uncased_128_mrpc  msr_paraphrase_test.txt	 train.tsv
dev_ids.tsv			       msr_paraphrase_train.txt
dev.tsv				       test.tsv


# 2 Fine-tune the BERT model

The spirit of BERT is to pre-train the language representations and then to fine-tune the deep bi-directional representations on a wide range of tasks with minimal task-dependent parameters, and achieves state-of-the-art results. In this tutorial, we will focus on fine-tuning with the pre-trained BERT model to classify semantically equivalent sentence pairs on MRPC task.

To fine-tune the pre-trained BERT model ("bert-base-uncased" model in HuggingFace transformers) for the MRPC task, you can follow the command in [examples](https://github.com/huggingface/transformers/tree/master/examples):
    
    export GLUE_DIR=./glue_data
    export TASK_NAME=MRPC
    export OUT_DIR=./$TASK_NAME/
    python ./run_glue.py \
        --model_type bert \
        --model_name_or_path bert-base-uncased \
        --task_name $TASK_NAME \
        --do_train \
        --do_eval \
        --do_lower_case \
        --data_dir $GLUE_DIR/$TASK_NAME \
        --max_seq_length 128 \
        --per_gpu_eval_batch_size=8   \
        --per_gpu_train_batch_size=8   \
        --learning_rate 2e-5 \
        --num_train_epochs 3.0 \
        --save_steps 100000 \
        --output_dir $OUT_DIR


We provide the fined-tuned BERT model for MRPC task [here](https://download.pytorch.org/tutorial/MRPC.zip).
To save time, you can download the model file (~400 MB) directly into your local folder ``$OUT_DIR``.



In [4]:
# %wget https://download.pytorch.org/tutorial/MRPC.zip
#%unzip MRPC.zip
%ls
%pwd

 bert_quant.pt                                        glue_data/
 bert_quant_state.pt                                  MRPC/
 download_glue_data.py                                read_model.ipynb
'(experimental)_Dynamic_Quantization_on_BERT.ipynb'


'/home/ty6232/DynamicQuantization_Bert'

## 2.1 Set global configurations

Here we set the global configurations for evaluating the fine-tuned BERT model before and after the dynamic quantization.

In [5]:
configs = Namespace()

# The output directory for the fine-tuned model.
configs.output_dir = "./MRPC/"

# The data directory for the MRPC task in the GLUE benchmark.
configs.data_dir = "./glue_data/MRPC"

# The model name or path for the pre-trained model.
configs.model_name_or_path = "bert-base-uncased"
# The maximum length of an input sequence
configs.max_seq_length = 128

# Prepare GLUE task.
configs.task_name = "MRPC".lower()
configs.processor = processors[configs.task_name]()
configs.output_mode = output_modes[configs.task_name]
configs.label_list = configs.processor.get_labels()
configs.model_type = "bert".lower()
configs.do_lower_case = True

# Set the device, batch size, topology, and caching flags.
configs.device = "cpu"
configs.per_gpu_eval_batch_size = 8
configs.n_gpu = 0
configs.local_rank = -1
configs.overwrite_cache = False


# Set random seed for reproducibility.
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
set_seed(42)

/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/data/processors/glue.py:174: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


## 2.2 Load the fine-tuned BERT model 

We load the tokenizer and fine-tuned BERT sequence classifier model (FP32) from the `configs.output_dir`.

In [6]:
tokenizer = BertTokenizer.from_pretrained(
    configs.output_dir, do_lower_case=configs.do_lower_case)

model = BertForSequenceClassification.from_pretrained(configs.output_dir)
model.to(configs.device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## 2.3 Define the tokenize and evaluation function
We reuse the tokenize and evaluation function from [HuggingFace](https://github.com/huggingface/transformers/blob/master/examples/run_glue.py).

In [7]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

def evaluate(args, model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_task_names = ("mnli", "mnli-mm") if args.task_name == "mnli" else (args.task_name,)
    eval_outputs_dirs = (args.output_dir, args.output_dir + '-MM') if args.task_name == "mnli" else (args.output_dir,)

    results = {}
    for eval_task, eval_output_dir in zip(eval_task_names, eval_outputs_dirs):
        eval_dataset = load_and_cache_examples(args, eval_task, tokenizer, evaluate=True)

        if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(eval_output_dir)

        args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
        # Note that DistributedSampler samples randomly
        eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

        # multi-gpu eval
        if args.n_gpu > 1:
            model = torch.nn.DataParallel(model)

        # Eval!
        logger.info("***** Running evaluation {} *****".format(prefix))
        logger.info("  Num examples = %d", len(eval_dataset))
        logger.info("  Batch size = %d", args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            batch = tuple(t.to(args.device) for t in batch)

            with torch.no_grad():
                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'labels':         batch[3]}
                if args.model_type != 'distilbert':
                    inputs['token_type_ids'] = batch[2] if args.model_type in ['bert', 'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs['labels'].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        if args.output_mode == "classification":
            preds = np.argmax(preds, axis=1)
        elif args.output_mode == "regression":
            preds = np.squeeze(preds)
        result = compute_metrics(eval_task, preds, out_label_ids)
        results.update(result)

        output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results {} *****".format(prefix))
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

    return results


def load_and_cache_examples(args, task, tokenizer, evaluate=False):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    processor = processors[task]()
    output_mode = output_modes[task]
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(args.data_dir, 'cached_{}_{}_{}_{}'.format(
        'dev' if evaluate else 'train',
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(args.max_seq_length),
        str(task)))
    if os.path.exists(cached_features_file) and not args.overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()
        if task in ['mnli', 'mnli-mm'] and args.model_type in ['roberta']:
            # HACK(label indices are swapped in RoBERTa pretrained model)
            label_list[1], label_list[2] = label_list[2], label_list[1] 
        examples = processor.get_dev_examples(args.data_dir) if evaluate else processor.get_train_examples(args.data_dir)
        features = convert_examples_to_features(examples,
                                                tokenizer,
                                                label_list=label_list,
                                                max_length=args.max_seq_length,
                                                output_mode=output_mode,
                                                task=None,
                                                # pad_on_left=bool(args.model_type in ['xlnet']),                 # pad on the left for xlnet
                                                # pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                # pad_token_segment_id=4 if args.model_type in ['xlnet'] else 0,
        )
        if args.local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)

    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
    return dataset


# 3.Apply the dynamic quantization

We call `torch.quantization.quantize_dynamic` on the model to apply the dynamic quantization on the HuggingFace BERT model. Specifically,

- We specify that we want the torch.nn.Linear modules in our model to be quantized;
- We specify that we want weights to be converted to quantized int8 values.

In [8]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (key): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (value): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (dropout): Dropout(p=0.1, inplace=False)
            )
      

## 作业要求：读取模型参数分布

In [9]:
y = torch.load("./bert_quant.pt")
print(y)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (key): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (value): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (dropout): Dropout(p=0.1, inplace=False)
            )
      

In [52]:
for name, param in quantized_model.named_parameters():
    print(name,param.data)

bert.embeddings.word_embeddings.weight tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]])
bert.embeddings.position_embeddings.weight tensor([[ 0.0174, -0.0258, -0.0356,  ...,  0.0004,  0.0004,  0.0154],
        [ 0.0075,  0.0024, -0.0199,  ...,  0.0294,  0.0298, -0.0052],
        [-0.0116, -0.0018, -0.0119,  ...,  0.0145,  0.0187, -0.0071],
        ...,
        [ 0.0174,  0.0035, -0.0096,  ...,  0.0030,  0.0004, -0.0269],
        [ 0.0217, -0.0060,  0.0147,  ..., -0.0056, -0.0126, -0.0281],
        [ 0.0026, -0.0233,  0.0055,  ...,  0.0175,  0.0275, -0.0777]])
bert.embeddings.token_type_embeddings.weight tenso

In [10]:
for name in quantized_model.state_dict():
   print(name)

bert.embeddings.position_ids
bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.scale
bert.encoder.layer.0.attention.self.query.zero_point
bert.encoder.layer.0.attention.self.query._packed_params.dtype
bert.encoder.layer.0.attention.self.query._packed_params._packed_params
bert.encoder.layer.0.attention.self.key.scale
bert.encoder.layer.0.attention.self.key.zero_point
bert.encoder.layer.0.attention.self.key._packed_params.dtype
bert.encoder.layer.0.attention.self.key._packed_params._packed_params
bert.encoder.layer.0.attention.self.value.scale
bert.encoder.layer.0.attention.self.value.zero_point
bert.encoder.layer.0.attention.self.value._packed_params.dtype
bert.encoder.layer.0.attention.self.value._packed_params._packed_params
bert.encoder.layer.0.attention.output.dense.scale
bert.encoder.layer.0.attention

In [57]:
from torch.utils.tensorboard import SummaryWriter
# strat training
writer = SummaryWriter(log_dir='./log')
# for epoch in range(3):
    # 【关键代码】
for name, param in quantized_model.named_parameters():
    # writer.add_histogram(tag=name+'_grad', values=param.grad, global_step=epoch)
    writer.add_histogram(tag=name+'_data', values=param.data)   
writer.close()


In [39]:
print(model.state_dict()['bert.encoder.layer.0.attention.self.key.weight'])
print(model.state_dict()['bert.encoder.layer.0.attention.self.key.bias'])
print(quantized_model.state_dict()['bert.encoder.layer.0.attention.self.key._packed_params._packed_params'])

tensor([[ 0.0074, -0.0374,  0.0197,  ...,  0.0147, -0.0137, -0.0093],
        [ 0.0265, -0.0637,  0.0174,  ..., -0.0171, -0.0423,  0.0172],
        [-0.0186, -0.0179, -0.0337,  ..., -0.0272, -0.0430, -0.0101],
        ...,
        [ 0.0192, -0.0188, -0.0108,  ...,  0.0179, -0.0570,  0.0843],
        [-0.0544,  0.0226, -0.0185,  ..., -0.0077, -0.0193, -0.0663],
        [ 0.0056, -0.0508,  0.0217,  ..., -0.0056, -0.0031, -0.0014]])
tensor([ 1.0426e-03,  3.5118e-04, -2.4630e-03, -1.0539e-04, -1.1933e-03,
         2.6931e-03,  2.2764e-03,  8.4061e-04,  2.2366e-03,  8.5655e-04,
         9.5801e-04,  1.4065e-03,  6.3340e-04, -1.6664e-03, -3.9759e-03,
         1.9305e-03,  2.1233e-04, -6.6227e-03, -9.6394e-04, -1.3682e-03,
         2.8621e-03,  3.8696e-03, -5.1299e-04,  8.7619e-04,  5.8972e-04,
        -3.4488e-03, -1.0479e-03, -2.9242e-04, -3.7488e-03, -1.1196e-02,
        -3.0306e-03, -9.7083e-04, -1.7843e-03,  1.2601e-03,  1.1208e-03,
         1.1752e-03, -5.4192e-04, -3.1822e-03,  2.5914e

## 3.1 Check the model size
Let's first check the model size. We can observe a significant reduction in model size:

In [17]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 438.017325
Size (MB): 181.497157


The BERT model used in this tutorial (bert-base-uncased) has a vocabulary size V of 30522. With the embedding size of 768, the total size of the word embedding table is ~ 4 (Bytes/FP32) * 30522 * 768 = 90 MB. So with the help of quantization, the model size of the non-embedding table part is reduced from 350 MB (FP32 model) to 90 MB (INT8 model).

## 3.2 Evaluate the inference accuracy and time

Next, let's compare the inference time as well as the evaluation accuracy between the original FP32 model and the INT8 model after the dynamic quantization.

In [18]:
def time_model_evaluation(model, configs, tokenizer):
    eval_start_time = time.time()
    result = evaluate(configs, model, tokenizer, prefix="")
    eval_end_time = time.time()
    eval_duration_time = eval_end_time - eval_start_time
    print(result)
    print("Evaluate total time (seconds): {0:.1f}".format(eval_duration_time))

# Evaluate the original FP32 BERT model
time_model_evaluation(model, configs, tokenizer)

Evaluating: 100%|██████████| 51/51 [01:00<00:00,  1.19s/it]

{'acc': 0.8602941176470589, 'f1': 0.9018932874354562, 'acc_and_f1': 0.8810937025412575}
Evaluate total time (seconds): 60.8



/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/data/metrics/__init__.py:42: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/data/metrics/__init__.py:36: FutureWarnin

In [19]:
# Evaluate the INT8 BERT model after the dynamic quantization
time_model_evaluation(quantized_model, configs, tokenizer)

/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/data/processors/glue.py:174: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)
Evaluating: 100%|██████████| 51/51 [00:24<00:00,  2.07it/s]

{'acc': 0.8553921568627451, 'f1': 0.8977469670710572, 'acc_and_f1': 0.8765695619669012}
Evaluate total time (seconds): 24.7



/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/data/metrics/__init__.py:42: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/data/metrics/__init__.py:36: FutureWarnin

Running this locally on a MacBook Pro, without quantization, inference (for all 408 examples in MRPC dataset) takes about 160 seconds, and with quantization it takes just about 90 seconds. We summarize the results for running the quantized BERT model inference on a Macbook Pro as the follows:

```
| Prec | F1 score | Model Size | 1 thread | 4 threads | 
| FP32 |  0.9019  |   438 MB   | 160 sec  | 85 sec    |
| INT8 |  0.8953  |   181 MB   |  90 sec  | 46 sec    |
```

We have 0.6% F1 score accuracy after applying the post-training dynamic quantization on the fine-tuned BERT model on the MRPC task. As a comparison, in a [recent paper](https://arxiv.org/pdf/1910.06188.pdf) (Table 1), it achieved 0.8788 by applying the post-training dynamic quantization and 0.8956 by applying the quantization-aware training. The main difference is that we support the asymmetric quantization in PyTorch while that paper supports the symmetric quantization only.

Note that we set the number of threads to 1 for the single-thread comparison in this tutorial. We also support the intra-op parallelization for these quantized INT8 operators. The users can now set multi-thread by `torch.set_num_threads(N)` (`N` is the number of intra-op parallelization threads). One preliminary requirement to enable the intra-op parallelization support is to build PyTorch with the right [backend](https://pytorch.org/docs/stable/notes/cpu_threading_torchscript_inference.html#build-options) such as OpenMP, Native, or TBB. You can use `torch.__config__.parallel_info()` to check the parallelization settings. On the same MacBook Pro using PyTorch with Native backend for parallelization, we can get about 46 seconds for processing the evaluation of MRPC dataset.

## 3.3 Serialize the quantized model
We can serialize and save the quantized model for the future use.

In [18]:
quantized_output_dir = configs.output_dir + "quantized/"
if not os.path.exists(quantized_output_dir):
    os.makedirs(quantized_output_dir)
    quantized_model.save_pretrained(quantized_output_dir)

In [19]:
torch.__config__.parallel_info() 

'ATen/Parallel:\n\tat::get_num_threads() : 4\n\tat::get_num_interop_threads() : 20\nOpenMP 201511 (a.k.a. OpenMP 4.5)\n\tomp_get_max_threads() : 4\nIntel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n\tmkl_get_max_threads() : 4\nIntel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)\nstd::thread::hardware_concurrency() : 40\nEnvironment variables:\n\tOMP_NUM_THREADS : [not set]\n\tMKL_NUM_THREADS : [not set]\nATen parallel backend: OpenMP\n'

In [22]:
# input_ids = ids_tensor([8, 128], 2)
# token_type_ids = ids_tensor([8, 128], 2)
# attention_mask = ids_tensor([8, 128], vocab_size=2)
# dummy_input = (input_ids, attention_mask, token_type_ids)
# traced_model = torch.jit.trace(quantized_model, dummy_input)

To load the quantized model, we can use torch.jit.load

In [20]:
loaded_quantized_model = torch.jit.load("bert_traced_eager_quant.pt")

ValueError: The provided filename bert_traced_eager_quant.pt does not exist

### save model and model with only state

In [41]:
torch.save(quantized_model,"bert_quant.pt")

In [42]:
torch.save(quantized_model.state_dict(),"bert_quant_state.pt")

Mine analyse model

In [33]:
import torch

# define a floating point model
class M(torch.nn.Module):
    def __init__(self):
        super(M, self).__init__()
        self.fc = torch.nn.Linear(4, 4)

    def forward(self, x):
        x = self.fc(x)
        return x

# create a model instance
model_fp32 = M()
# create a quantized model instance
model_int8 = torch.quantization.quantize_dynamic(
    model_fp32,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights

print(model_fp32)
print(model_int8)
# run the model
input_fp32 = torch.randn(4, 4, 4, 4)
res = model_int8(input_fp32)

M(
  (fc): Linear(in_features=4, out_features=4, bias=True)
)
M(
  (fc): DynamicQuantizedLinear(in_features=4, out_features=4, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)


In [34]:
import torch
import torchvision
from thop import profile
 
dummy_input = torch.randn(1, 3, 224, 224)
flops, params = profile(quantized_model, (dummy_input,))
print('FLOPs: ', flops, 'params: ', params)

ModuleNotFoundError: No module named 'thop'

# Conclusion
In this tutorial, we demonstrated how to demonstrate how to convert a well-known state-of-the-art NLP model like BERT into dynamic quantized model. Dynamic quantization can reduce the size of the model while only having a limited implication on accuracy.

Thanks for reading! As always, we welcome any feedback, so please create an issue [here](https://github.com/pytorch/pytorch/issues) if you have any.

# References
[1] J.Devlin, M. Chang, K. Lee and K. Toutanova, [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf) (2018)

[2] [HuggingFace Transformers](https://github.com/huggingface/transformers).

[3] O. Zafrir, G. Boudoukh, P. Izsak, & M. Wasserblat (2019). [Q8BERT: Quantized 8bit BERT](https://arxiv.org/pdf/1910.06188.pdf).

